In [ ]:
import os
import pandas as pd
import pandas_ta as ta
from math import pi
from bokeh.plotting import figure, show
import joblib

In [ ]:
MINIMUM_TIME = 930
MAXIMUM_TIME = 1430

MINIMUM_DATE_DATAFRAME = 20200401
MINIMUM_DATE_TRADE = 20200501

MAX_TRAIN_DATE = 20210831

CURRENT_TARGET = 0.4
CURRENT_STOP = 0.2

CURRENT_ASSET = 'WDO'
CURRENT_TIMEFRAME = '5Min'

CURRENT_DAILY_FILE_CSV = 'WDO$Daily.csv'
CURRENT_5MIN_FILE_CSV = 'WDO$M5.csv'

In [ ]:
csv_diario = pd.read_csv(os.path.join('..','00.data','input',CURRENT_DAILY_FILE_CSV))
csv_5Min = pd.read_csv(os.path.join('..','00.data','input',CURRENT_5MIN_FILE_CSV))

csv_diario['Date'] = csv_diario.apply(lambda x: int(x['datetime'].replace('.','')), axis=1)
csv_5Min['Date'] = csv_5Min.apply(lambda x: int(x['datetime'].split(' ')[0].replace('.','')), axis=1)
csv_5Min['Time'] = csv_5Min.apply(lambda x: int(x['datetime'].split(' ')[1].replace(':','')), axis=1)

csv_diario = csv_diario[csv_diario['Date'] >= MINIMUM_DATE_DATAFRAME]
csv_5Min = csv_5Min[csv_5Min['Date'] >= MINIMUM_DATE_DATAFRAME]

csv_5Min['ema9'] = csv_5Min.ta.ema(length=9)
csv_5Min['ema21'] = csv_5Min.ta.ema(length=21)

#csv_5Min.dtypes
#csv_5Min.head()

In [ ]:
data_daily = csv_diario.to_dict('records')
data_5min = csv_5Min.to_dict('records')
daily_dates = []

ohlc_daily = {}

for current_date in data_daily:
    ohlc_daily[current_date['Date']] = current_date
    daily_dates.append(current_date['Date'])

#print(data_5min[:15])
#print(daily_dates)


In [ ]:
def is_short(current_close, bars, target, stop):
    correct_target = current_close - target
    correct_stop = current_close + stop
    
    for x in bars:
        if x['high'] >= correct_stop:
            return False
        if x['low'] < correct_target:
            return True
    return False
    
def is_long(current_close, bars, target, stop):
    correct_target = current_close + target
    correct_stop = current_close - stop
    
    for x in bars:
        if x['high'] >= correct_target:
            return True
        if x['low'] < correct_stop:
            return False
    return False


In [ ]:
generated_bars_train = []
generated_bars_test = []

for index, _ in enumerate(data_5min):
    current_date = data_5min[index]['Date']
    current_time = data_5min[index]['Time']
    
    if current_time < MINIMUM_TIME or \
        current_time > MAXIMUM_TIME or \
        current_date < MINIMUM_DATE_TRADE:
        continue
    
    if current_date > MAX_TRAIN_DATE:
        generated_bars = generated_bars_test
    else:
        generated_bars = generated_bars_train
    
    current_open = data_5min[index]['open']
    current_volume = data_5min[index]['volume']
    
    previous_date = [x for x in daily_dates if x < current_date]
    previous_date = previous_date[len(previous_date)-1]
    previous_date = ohlc_daily[previous_date]
    
    current_bar = {}
    
    current_bar['current_open'] = current_open
    current_bar['current_date'] = current_date
    current_bar['current_time'] = current_time
    
    current_bar['previous_date'] = previous_date['Date']
    current_bar['previous_high_real'] = previous_date['high']
    current_bar['previous_close_real'] = previous_date['close']
    current_bar['previous_low_real'] = previous_date['low']
    current_bar['previous_open_real'] = previous_date['open']    
    current_bar['previous_high'] = ((previous_date['high'] / current_open) - 1) * 100
    current_bar['previous_close'] = ((previous_date['close'] / current_open) - 1) * 100
    current_bar['previous_low'] = ((previous_date['low'] / current_open) - 1) * 100
    current_bar['previous_open'] =((previous_date['open'] / current_open) - 1) * 100
    
    
    current_bar['x0_ema9'] = ((data_5min[index-11]['ema9'] / current_open) - 1) * 100
    current_bar['x0_ema21'] = ((data_5min[index-11]['ema21'] / current_open) - 1) * 100
    current_bar['x0_open'] = ((data_5min[index-11]['open'] / current_open) - 1) * 100
    current_bar['x0_high'] = ((data_5min[index-11]['high'] / current_open) - 1) * 100
    current_bar['x0_low'] = ((data_5min[index-11]['low'] / current_open) - 1) * 100
    current_bar['x0_close'] = ((data_5min[index-11]['close'] / current_open) - 1) * 100
    current_bar['x0_volume'] = ((data_5min[index-11]['volume'] / current_volume) - 1) * 100
    current_bar['x0_open_real'] = data_5min[index-11]['open'] 
    current_bar['x0_high_real'] = data_5min[index-11]['high'] 
    current_bar['x0_low_real'] = data_5min[index-11]['low'] 
    current_bar['x0_close_real'] = data_5min[index-11]['close'] 

    
    current_bar['x1_ema9'] = ((data_5min[index-10]['ema9'] / current_open) - 1) * 100
    current_bar['x1_ema21'] = ((data_5min[index-10]['ema21'] / current_open) - 1) * 100
    current_bar['x1_open'] = ((data_5min[index-10]['open'] / current_open) - 1) * 100
    current_bar['x1_high'] = ((data_5min[index-10]['high'] / current_open) - 1) * 100
    current_bar['x1_low'] = ((data_5min[index-10]['low'] / current_open) - 1) * 100
    current_bar['x1_close'] = ((data_5min[index-10]['close'] / current_open) - 1) * 100
    current_bar['x1_volume'] = ((data_5min[index-10]['volume'] / current_volume) - 1) * 100
    current_bar['x1_open_real'] = data_5min[index-10]['open'] 
    current_bar['x1_high_real'] = data_5min[index-10]['high'] 
    current_bar['x1_low_real'] = data_5min[index-10]['low'] 
    current_bar['x1_close_real'] = data_5min[index-10]['close'] 

    
    current_bar['x2_ema9'] = ((data_5min[index-9]['ema9'] / current_open) - 1) * 100
    current_bar['x2_ema21'] = ((data_5min[index-9]['ema21'] / current_open) - 1) * 100
    current_bar['x2_open'] = ((data_5min[index-9]['open'] / current_open) - 1) * 100
    current_bar['x2_high'] = ((data_5min[index-9]['high'] / current_open) - 1) * 100
    current_bar['x2_low'] = ((data_5min[index-9]['low'] / current_open) - 1) * 100
    current_bar['x2_close'] = ((data_5min[index-9]['close'] / current_open) - 1) * 100
    current_bar['x2_volume'] = ((data_5min[index-9]['volume'] / current_volume) - 1) * 100    
    current_bar['x2_open_real'] = data_5min[index-9]['open'] 
    current_bar['x2_high_real'] = data_5min[index-9]['high'] 
    current_bar['x2_low_real'] = data_5min[index-9]['low'] 
    current_bar['x2_close_real'] = data_5min[index-9]['close']

    
    current_bar['x3_ema9'] = ((data_5min[index-8]['ema9'] / current_open) - 1) * 100
    current_bar['x3_ema21'] = ((data_5min[index-8]['ema21'] / current_open) - 1) * 100
    current_bar['x3_open'] = ((data_5min[index-8]['open'] / current_open) - 1) * 100
    current_bar['x3_high'] = ((data_5min[index-8]['high'] / current_open) - 1) * 100
    current_bar['x3_low'] = ((data_5min[index-8]['low'] / current_open) - 1) * 100
    current_bar['x3_close'] = ((data_5min[index-8]['close'] / current_open) - 1) * 100
    current_bar['x3_volume'] = ((data_5min[index-8]['volume'] / current_volume) - 1) * 100        
    current_bar['x3_open_real'] = data_5min[index-8]['open'] 
    current_bar['x3_high_real'] = data_5min[index-8]['high'] 
    current_bar['x3_low_real'] = data_5min[index-8]['low'] 
    current_bar['x3_close_real'] = data_5min[index-8]['close']

    
    current_bar['x4_ema9'] = ((data_5min[index-7]['ema9'] / current_open) - 1) * 100
    current_bar['x4_ema21'] = ((data_5min[index-7]['ema21'] / current_open) - 1) * 100
    current_bar['x4_open'] = ((data_5min[index-7]['open'] / current_open) - 1) * 100
    current_bar['x4_high'] = ((data_5min[index-7]['high'] / current_open) - 1) * 100
    current_bar['x4_low'] = ((data_5min[index-7]['low'] / current_open) - 1) * 100
    current_bar['x4_close'] = ((data_5min[index-7]['close'] / current_open) - 1) * 100
    current_bar['x4_volume'] = ((data_5min[index-7]['volume'] / current_volume) - 1) * 100            
    current_bar['x4_open_real'] = data_5min[index-7]['open'] 
    current_bar['x4_high_real'] = data_5min[index-7]['high'] 
    current_bar['x4_low_real'] = data_5min[index-7]['low'] 
    current_bar['x4_close_real'] = data_5min[index-7]['close'] 

    
    current_bar['x5_ema9'] = ((data_5min[index-6]['ema9'] / current_open) - 1) * 100
    current_bar['x5_ema21'] = ((data_5min[index-6]['ema21'] / current_open) - 1) * 100
    current_bar['x5_open'] = ((data_5min[index-6]['open'] / current_open) - 1) * 100
    current_bar['x5_high'] = ((data_5min[index-6]['high'] / current_open) - 1) * 100
    current_bar['x5_low'] = ((data_5min[index-6]['low'] / current_open) - 1) * 100
    current_bar['x5_close'] = ((data_5min[index-6]['close'] / current_open) - 1) * 100
    current_bar['x5_volume'] = ((data_5min[index-6]['volume'] / current_volume) - 1) * 100            
    current_bar['x5_open_real'] = data_5min[index-6]['open'] 
    current_bar['x5_high_real'] = data_5min[index-6]['high'] 
    current_bar['x5_low_real'] = data_5min[index-6]['low'] 
    current_bar['x5_close_real'] = data_5min[index-6]['close'] 

    current_bar['x6_ema9'] = ((data_5min[index-5]['ema9'] / current_open) - 1) * 100
    current_bar['x6_ema21'] = ((data_5min[index-5]['ema21'] / current_open) - 1) * 100
    current_bar['x6_open'] = ((data_5min[index-5]['open'] / current_open) - 1) * 100
    current_bar['x6_high'] = ((data_5min[index-5]['high'] / current_open) - 1) * 100
    current_bar['x6_low'] = ((data_5min[index-5]['low'] / current_open) - 1) * 100
    current_bar['x6_close'] = ((data_5min[index-5]['close'] / current_open) - 1) * 100
    current_bar['x6_volume'] = ((data_5min[index-5]['volume'] / current_volume) - 1) * 100            
    current_bar['x6_open_real'] = data_5min[index-5]['open'] 
    current_bar['x6_high_real'] = data_5min[index-5]['high'] 
    current_bar['x6_low_real'] = data_5min[index-5]['low'] 
    current_bar['x6_close_real'] = data_5min[index-5]['close'] 

    current_bar['x7_ema9'] = ((data_5min[index-4]['ema9'] / current_open) - 1) * 100
    current_bar['x7_ema21'] = ((data_5min[index-4]['ema21'] / current_open) - 1) * 100
    current_bar['x7_open'] = ((data_5min[index-4]['open'] / current_open) - 1) * 100
    current_bar['x7_high'] = ((data_5min[index-4]['high'] / current_open) - 1) * 100
    current_bar['x7_low'] = ((data_5min[index-4]['low'] / current_open) - 1) * 100
    current_bar['x7_close'] = ((data_5min[index-4]['close'] / current_open) - 1) * 100
    current_bar['x7_volume'] = ((data_5min[index-4]['volume'] / current_volume) - 1) * 100            
    current_bar['x7_open_real'] = data_5min[index-4]['open'] 
    current_bar['x7_high_real'] = data_5min[index-4]['high'] 
    current_bar['x7_low_real'] = data_5min[index-4]['low'] 
    current_bar['x7_close_real'] = data_5min[index-4]['close'] 
    
    current_bar['x8_ema9'] = ((data_5min[index-3]['ema9'] / current_open) - 1) * 100
    current_bar['x8_ema21'] = ((data_5min[index-3]['ema21'] / current_open) - 1) * 100
    current_bar['x8_open'] = ((data_5min[index-3]['open'] / current_open) - 1) * 100
    current_bar['x8_high'] = ((data_5min[index-3]['high'] / current_open) - 1) * 100
    current_bar['x8_low'] = ((data_5min[index-3]['low'] / current_open) - 1) * 100
    current_bar['x8_close'] = ((data_5min[index-3]['close'] / current_open) - 1) * 100
    current_bar['x8_volume'] = ((data_5min[index-3]['volume'] / current_volume) - 1) * 100            
    current_bar['x8_open_real'] = data_5min[index-3]['open'] 
    current_bar['x8_high_real'] = data_5min[index-3]['high'] 
    current_bar['x8_low_real'] = data_5min[index-3]['low'] 
    current_bar['x8_close_real'] = data_5min[index-3]['close']     

    current_bar['x9_ema9'] = ((data_5min[index-2]['ema9'] / current_open) - 1) * 100
    current_bar['x9_ema21'] = ((data_5min[index-2]['ema21'] / current_open) - 1) * 100
    current_bar['x9_open'] = ((data_5min[index-2]['open'] / current_open) - 1) * 100
    current_bar['x9_high'] = ((data_5min[index-2]['high'] / current_open) - 1) * 100
    current_bar['x9_low'] = ((data_5min[index-2]['low'] / current_open) - 1) * 100
    current_bar['x9_close'] = ((data_5min[index-2]['close'] / current_open) - 1) * 100
    current_bar['x9_volume'] = ((data_5min[index-2]['volume'] / current_volume) - 1) * 100            
    current_bar['x9_open_real'] = data_5min[index-2]['open'] 
    current_bar['x9_high_real'] = data_5min[index-2]['high'] 
    current_bar['x9_low_real'] = data_5min[index-2]['low'] 
    current_bar['x9_close_real'] = data_5min[index-2]['close']     

    current_bar['x10_ema9'] = ((data_5min[index-1]['ema9'] / current_open) - 1) * 100
    current_bar['x10_ema21'] = ((data_5min[index-1]['ema21'] / current_open) - 1) * 100
    current_bar['x10_open'] = ((data_5min[index-1]['open'] / current_open) - 1) * 100
    current_bar['x10_high'] = ((data_5min[index-1]['high'] / current_open) - 1) * 100
    current_bar['x10_low'] = ((data_5min[index-1]['low'] / current_open) - 1) * 100
    current_bar['x10_close'] = ((data_5min[index-1]['close'] / current_open) - 1) * 100
    current_bar['x10_volume'] = ((data_5min[index-1]['volume'] / current_volume) - 1) * 100            
    current_bar['x10_open_real'] = data_5min[index-1]['open'] 
    current_bar['x10_high_real'] = data_5min[index-1]['high'] 
    current_bar['x10_low_real'] = data_5min[index-1]['low'] 
    current_bar['x10_close_real'] = data_5min[index-1]['close']         
    
    current_bar['x11_ema9'] = ((data_5min[index]['ema9'] / current_open) - 1) * 100
    current_bar['x11_ema21'] = ((data_5min[index]['ema21'] / current_open) - 1) * 100
    current_bar['x11_open'] = ((data_5min[index]['open'] / current_open) - 1) * 100
    current_bar['x11_high'] = ((data_5min[index]['high'] / current_open) - 1) * 100
    current_bar['x11_low'] = ((data_5min[index]['low'] / current_open) - 1) * 100
    current_bar['x11_close'] = ((data_5min[index]['close'] / current_open) - 1) * 100
    current_bar['x11_volume'] = ((data_5min[index]['volume'] / current_volume) - 1) * 100            
    current_bar['x11_open_real'] = data_5min[index]['open'] 
    current_bar['x11_high_real'] = data_5min[index]['high'] 
    current_bar['x11_low_real'] = data_5min[index]['low'] 
    current_bar['x11_close_real'] = data_5min[index]['close']             
    
    
    
    
    current_bar['y0_ema9'] = ((data_5min[index+1]['ema9'] / current_open) - 1) * 100
    current_bar['y0_ema21'] = ((data_5min[index+1]['ema21'] / current_open) - 1) * 100
    current_bar['y0_open'] = ((data_5min[index+1]['open'] / current_open) - 1) * 100
    current_bar['y0_high'] = ((data_5min[index+1]['high'] / current_open) - 1) * 100
    current_bar['y0_low'] = ((data_5min[index+1]['low'] / current_open) - 1) * 100
    current_bar['y0_close'] = ((data_5min[index+1]['close'] / current_open) - 1) * 100
    current_bar['y0_volume'] = ((data_5min[index+1]['volume'] / current_volume) - 1) * 100
    current_bar['y0_open_real'] = data_5min[index+1]['open'] 
    current_bar['y0_high_real'] = data_5min[index+1]['high'] 
    current_bar['y0_low_real'] = data_5min[index+1]['low'] 
    current_bar['y0_close_real'] = data_5min[index+1]['close'] 

    
    current_bar['y1_ema9'] = ((data_5min[index+2]['ema9'] / current_open) - 1) * 100
    current_bar['y1_ema21'] = ((data_5min[index+2]['ema21'] / current_open) - 1) * 100
    current_bar['y1_open'] = ((data_5min[index+2]['open'] / current_open) - 1) * 100
    current_bar['y1_high'] = ((data_5min[index+2]['high'] / current_open) - 1) * 100
    current_bar['y1_low'] = ((data_5min[index+2]['low'] / current_open) - 1) * 100
    current_bar['y1_close'] = ((data_5min[index+2]['close'] / current_open) - 1) * 100
    current_bar['y1_volume'] = ((data_5min[index+2]['volume'] / current_volume) - 1) * 100
    current_bar['y1_open_real'] = data_5min[index+2]['open'] 
    current_bar['y1_high_real'] = data_5min[index+2]['high'] 
    current_bar['y1_low_real'] = data_5min[index+2]['low'] 
    current_bar['y1_close_real'] = data_5min[index+2]['close'] 
    
    current_bar['y2_ema9'] = ((data_5min[index+3]['ema9'] / current_open) - 1) * 100
    current_bar['y2_ema21'] = ((data_5min[index+3]['ema21'] / current_open) - 1) * 100
    current_bar['y2_open'] = ((data_5min[index+3]['open'] / current_open) - 1) * 100
    current_bar['y2_high'] = ((data_5min[index+3]['high'] / current_open) - 1) * 100
    current_bar['y2_low'] = ((data_5min[index+3]['low'] / current_open) - 1) * 100
    current_bar['y2_close'] = ((data_5min[index+3]['close'] / current_open) - 1) * 100
    current_bar['y2_volume'] = ((data_5min[index+3]['volume'] / current_volume) - 1) * 100    
    current_bar['y2_open_real'] = data_5min[index+3]['open'] 
    current_bar['y2_high_real'] = data_5min[index+3]['high'] 
    current_bar['y2_low_real'] = data_5min[index+3]['low'] 
    current_bar['y2_close_real'] = data_5min[index+3]['close'] 

    
    current_bar['y3_ema9'] = ((data_5min[index+4]['ema9'] / current_open) - 1) * 100
    current_bar['y3_ema21'] = ((data_5min[index+4]['ema21'] / current_open) - 1) * 100
    current_bar['y3_open'] = ((data_5min[index+4]['open'] / current_open) - 1) * 100
    current_bar['y3_high'] = ((data_5min[index+4]['high'] / current_open) - 1) * 100
    current_bar['y3_low'] = ((data_5min[index+4]['low'] / current_open) - 1) * 100
    current_bar['y3_close'] = ((data_5min[index+4]['close'] / current_open) - 1) * 100
    current_bar['y3_volume'] = ((data_5min[index+4]['volume'] / current_volume) - 1) * 100        
    current_bar['y3_open_real'] = data_5min[index+4]['open'] 
    current_bar['y3_high_real'] = data_5min[index+4]['high'] 
    current_bar['y3_low_real'] = data_5min[index+4]['low'] 
    current_bar['y3_close_real'] = data_5min[index+4]['close']

    
    current_bar['y4_ema9'] = ((data_5min[index+5]['ema9'] / current_open) - 1) * 100
    current_bar['y4_ema21'] = ((data_5min[index+5]['ema21'] / current_open) - 1) * 100
    current_bar['y4_open'] = ((data_5min[index+5]['open'] / current_open) - 1) * 100
    current_bar['y4_high'] = ((data_5min[index+5]['high'] / current_open) - 1) * 100
    current_bar['y4_low'] = ((data_5min[index+5]['low'] / current_open) - 1) * 100
    current_bar['y4_close'] = ((data_5min[index+5]['close'] / current_open) - 1) * 100
    current_bar['y4_volume'] = ((data_5min[index+5]['volume'] / current_volume) - 1) * 100            
    current_bar['y4_open_real'] = data_5min[index+5]['open'] 
    current_bar['y4_high_real'] = data_5min[index+5]['high'] 
    current_bar['y4_low_real'] = data_5min[index+5]['low'] 
    current_bar['y4_close_real'] = data_5min[index+5]['close'] 

    current_bar['y5_ema9'] = ((data_5min[index+6]['ema9'] / current_open) - 1) * 100
    current_bar['y5_ema21'] = ((data_5min[index+6]['ema21'] / current_open) - 1) * 100
    current_bar['y5_open'] = ((data_5min[index+6]['open'] / current_open) - 1) * 100
    current_bar['y5_high'] = ((data_5min[index+6]['high'] / current_open) - 1) * 100
    current_bar['y5_low'] = ((data_5min[index+6]['low'] / current_open) - 1) * 100
    current_bar['y5_close'] = ((data_5min[index+6]['close'] / current_open) - 1) * 100
    current_bar['y5_volume'] = ((data_5min[index+6]['volume'] / current_volume) - 1) * 100                
    current_bar['y5_open_real'] = data_5min[index+6]['open'] 
    current_bar['y5_high_real'] = data_5min[index+6]['high']
    current_bar['y5_low_real'] = data_5min[index+6]['low']
    current_bar['y5_close_real'] = data_5min[index+6]['close']

    current_bar['y6_ema9'] = ((data_5min[index+7]['ema9'] / current_open) - 1) * 100
    current_bar['y6_ema21'] = ((data_5min[index+7]['ema21'] / current_open) - 1) * 100
    current_bar['y6_open'] = ((data_5min[index+7]['open'] / current_open) - 1) * 100
    current_bar['y6_high'] = ((data_5min[index+7]['high'] / current_open) - 1) * 100
    current_bar['y6_low'] = ((data_5min[index+7]['low'] / current_open) - 1) * 100
    current_bar['y6_close'] = ((data_5min[index+7]['close'] / current_open) - 1) * 100
    current_bar['y6_volume'] = ((data_5min[index+7]['volume'] / current_volume) - 1) * 100                
    current_bar['y6_open_real'] = data_5min[index+7]['open'] 
    current_bar['y6_high_real'] = data_5min[index+7]['high']
    current_bar['y6_low_real'] = data_5min[index+7]['low']
    current_bar['y6_close_real'] = data_5min[index+7]['close']

    current_bar['y7_ema9'] = ((data_5min[index+8]['ema9'] / current_open) - 1) * 100
    current_bar['y7_ema21'] = ((data_5min[index+8]['ema21'] / current_open) - 1) * 100
    current_bar['y7_open'] = ((data_5min[index+8]['open'] / current_open) - 1) * 100
    current_bar['y7_high'] = ((data_5min[index+8]['high'] / current_open) - 1) * 100
    current_bar['y7_low'] = ((data_5min[index+8]['low'] / current_open) - 1) * 100
    current_bar['y7_close'] = ((data_5min[index+8]['close'] / current_open) - 1) * 100
    current_bar['y7_volume'] = ((data_5min[index+8]['volume'] / current_volume) - 1) * 100                
    current_bar['y7_open_real'] = data_5min[index+8]['open'] 
    current_bar['y7_high_real'] = data_5min[index+8]['high']
    current_bar['y7_low_real'] = data_5min[index+8]['low']
    current_bar['y7_close_real'] = data_5min[index+8]['close']

    current_bar['y8_ema9'] = ((data_5min[index+9]['ema9'] / current_open) - 1) * 100
    current_bar['y8_ema21'] = ((data_5min[index+9]['ema21'] / current_open) - 1) * 100
    current_bar['y8_open'] = ((data_5min[index+9]['open'] / current_open) - 1) * 100
    current_bar['y8_high'] = ((data_5min[index+9]['high'] / current_open) - 1) * 100
    current_bar['y8_low'] = ((data_5min[index+9]['low'] / current_open) - 1) * 100
    current_bar['y8_close'] = ((data_5min[index+9]['close'] / current_open) - 1) * 100
    current_bar['y8_volume'] = ((data_5min[index+9]['volume'] / current_volume) - 1) * 100                
    current_bar['y8_open_real'] = data_5min[index+9]['open'] 
    current_bar['y8_high_real'] = data_5min[index+9]['high']
    current_bar['y8_low_real'] = data_5min[index+9]['low']
    current_bar['y8_close_real'] = data_5min[index+9]['close']
    
    current_bar['y9_ema9'] = ((data_5min[index+10]['ema9'] / current_open) - 1) * 100
    current_bar['y9_ema21'] = ((data_5min[index+10]['ema21'] / current_open) - 1) * 100
    current_bar['y9_open'] = ((data_5min[index+10]['open'] / current_open) - 1) * 100
    current_bar['y9_high'] = ((data_5min[index+10]['high'] / current_open) - 1) * 100
    current_bar['y9_low'] = ((data_5min[index+10]['low'] / current_open) - 1) * 100
    current_bar['y9_close'] = ((data_5min[index+10]['close'] / current_open) - 1) * 100
    current_bar['y9_volume'] = ((data_5min[index+10]['volume'] / current_volume) - 1) * 100                
    current_bar['y9_open_real'] = data_5min[index+10]['open'] 
    current_bar['y9_high_real'] = data_5min[index+10]['high']
    current_bar['y9_low_real'] = data_5min[index+10]['low']
    current_bar['y9_close_real'] = data_5min[index+10]['close']

    current_bar['y10_ema9'] = ((data_5min[index+11]['ema9'] / current_open) - 1) * 100
    current_bar['y10_ema21'] = ((data_5min[index+11]['ema21'] / current_open) - 1) * 100
    current_bar['y10_open'] = ((data_5min[index+11]['open'] / current_open) - 1) * 100
    current_bar['y10_high'] = ((data_5min[index+11]['high'] / current_open) - 1) * 100
    current_bar['y10_low'] = ((data_5min[index+11]['low'] / current_open) - 1) * 100
    current_bar['y10_close'] = ((data_5min[index+11]['close'] / current_open) - 1) * 100
    current_bar['y10_volume'] = ((data_5min[index+11]['volume'] / current_volume) - 1) * 100                
    current_bar['y10_open_real'] = data_5min[index+11]['open'] 
    current_bar['y10_high_real'] = data_5min[index+11]['high']
    current_bar['y10_low_real'] = data_5min[index+11]['low']
    current_bar['y10_close_real'] = data_5min[index+11]['close']    
    
    current_bar['y11_ema9'] = ((data_5min[index+12]['ema9'] / current_open) - 1) * 100
    current_bar['y11_ema21'] = ((data_5min[index+12]['ema21'] / current_open) - 1) * 100
    current_bar['y11_open'] = ((data_5min[index+12]['open'] / current_open) - 1) * 100
    current_bar['y11_high'] = ((data_5min[index+12]['high'] / current_open) - 1) * 100
    current_bar['y11_low'] = ((data_5min[index+12]['low'] / current_open) - 1) * 100
    current_bar['y11_close'] = ((data_5min[index+12]['close'] / current_open) - 1) * 100
    current_bar['y11_volume'] = ((data_5min[index+12]['volume'] / current_volume) - 1) * 100                
    current_bar['y11_open_real'] = data_5min[index+12]['open'] 
    current_bar['y11_high_real'] = data_5min[index+12]['high']
    current_bar['y11_low_real'] = data_5min[index+12]['low']
    current_bar['y11_close_real'] = data_5min[index+12]['close']        

    current_bar['chart_data'] = [
        {
        'date': 0,
        'open': current_bar['x0_open'],
        'high': current_bar['x0_high'],
        'low': current_bar['x0_low'],
        'close': current_bar['x0_close'],
        'ema9': current_bar['x0_ema9'],
        'ema21': current_bar['x0_ema21'],
        'open_real': current_bar['x0_open_real'],
        'high_real': current_bar['x0_high_real'],
        'low_real': current_bar['x0_low_real'],
        'close_real': current_bar['x0_close_real']
        },
        {
        'date': 1,
        'open': current_bar['x1_open'],
        'high': current_bar['x1_high'],
        'low': current_bar['x1_low'],
        'close': current_bar['x1_close'],
        'ema9': current_bar['x1_ema9'],
        'ema21': current_bar['x1_ema21'],
        'open_real': current_bar['x1_open_real'],
        'high_real': current_bar['x1_high_real'],
        'low_real': current_bar['x1_low_real'],
        'close_real': current_bar['x1_close_real']
        },
        {
        'date': 2,
        'open': current_bar['x2_open'],
        'high': current_bar['x2_high'],
        'low': current_bar['x2_low'],
        'close': current_bar['x2_close'],
        'ema9': current_bar['x2_ema9'],
        'ema21': current_bar['x2_ema21'],
        'open_real': current_bar['x2_open_real'],
        'high_real': current_bar['x2_high_real'],
        'low_real': current_bar['x2_low_real'],
        'close_real': current_bar['x2_close_real']
        },
        {
        'date': 3,
        'open': current_bar['x3_open'],
        'high': current_bar['x3_high'],
        'low': current_bar['x3_low'],
        'close': current_bar['x3_close'],
        'ema9': current_bar['x3_ema9'],
        'ema21': current_bar['x3_ema21'],
        'open_real': current_bar['x3_open_real'],
        'high_real': current_bar['x3_high_real'],
        'low_real': current_bar['x3_low_real'],
        'close_real': current_bar['x3_close_real']
        },     
        {
        'date': 4,
        'open': current_bar['x4_open'],
        'high': current_bar['x4_high'],
        'low': current_bar['x4_low'],
        'close': current_bar['x4_close'],
        'ema9': current_bar['x4_ema9'],
        'ema21': current_bar['x4_ema21'],
        'open_real': current_bar['x4_open_real'],
        'high_real': current_bar['x4_high_real'],
        'low_real': current_bar['x4_low_real'],
        'close_real': current_bar['x4_close_real']
        },             
        {
        'date': 5,
        'open': current_bar['x5_open'],
        'high': current_bar['x5_high'],
        'low': current_bar['x5_low'],
        'close': current_bar['x5_close'],
        'ema9': current_bar['x5_ema9'],
        'ema21': current_bar['x5_ema21'],
        'open_real': current_bar['x5_open_real'],
        'high_real': current_bar['x5_high_real'],
        'low_real': current_bar['x5_low_real'],
        'close_real': current_bar['x5_close_real']
        },
        {
        'date': 6,
        'open': current_bar['x6_open'],
        'high': current_bar['x6_high'],
        'low': current_bar['x6_low'],
        'close': current_bar['x6_close'],
        'ema9': current_bar['x6_ema9'],
        'ema21': current_bar['x6_ema21'],
        'open_real': current_bar['x6_open_real'],
        'high_real': current_bar['x6_high_real'],
        'low_real': current_bar['x6_low_real'],
        'close_real': current_bar['x6_close_real']
        },
        {
        'date': 7,
        'open': current_bar['x7_open'],
        'high': current_bar['x7_high'],
        'low': current_bar['x7_low'],
        'close': current_bar['x7_close'],
        'ema9': current_bar['x7_ema9'],
        'ema21': current_bar['x7_ema21'],
        'open_real': current_bar['x7_open_real'],
        'high_real': current_bar['x7_high_real'],
        'low_real': current_bar['x7_low_real'],
        'close_real': current_bar['x7_close_real']
        },
        {
        'date': 8,
        'open': current_bar['x8_open'],
        'high': current_bar['x8_high'],
        'low': current_bar['x8_low'],
        'close': current_bar['x8_close'],
        'ema9': current_bar['x8_ema9'],
        'ema21': current_bar['x8_ema21'],
        'open_real': current_bar['x8_open_real'],
        'high_real': current_bar['x8_high_real'],
        'low_real': current_bar['x8_low_real'],
        'close_real': current_bar['x8_close_real']
        },
        {
        'date': 9,
        'open': current_bar['x9_open'],
        'high': current_bar['x9_high'],
        'low': current_bar['x9_low'],
        'close': current_bar['x9_close'],
        'ema9': current_bar['x9_ema9'],
        'ema21': current_bar['x9_ema21'],
        'open_real': current_bar['x9_open_real'],
        'high_real': current_bar['x9_high_real'],
        'low_real': current_bar['x9_low_real'],
        'close_real': current_bar['x9_close_real']
        },        
        {
        'date': 10,
        'open': current_bar['x10_open'],
        'high': current_bar['x10_high'],
        'low': current_bar['x10_low'],
        'close': current_bar['x10_close'],
        'ema9': current_bar['x10_ema9'],
        'ema21': current_bar['x10_ema21'],
        'open_real': current_bar['x10_open_real'],
        'high_real': current_bar['x10_high_real'],
        'low_real': current_bar['x10_low_real'],
        'close_real': current_bar['x10_close_real']
        },        
        {
        'date': 11,
        'open': current_bar['x11_open'],
        'high': current_bar['x11_high'],
        'low': current_bar['x11_low'],
        'close': current_bar['x11_close'],
        'ema9': current_bar['x11_ema9'],
        'ema21': current_bar['x11_ema21'],
        'open_real': current_bar['x11_open_real'],
        'high_real': current_bar['x11_high_real'],
        'low_real': current_bar['x11_low_real'],
        'close_real': current_bar['x11_close_real']
        },                
        {
        'date': 12,
        'open': current_bar['y0_open'],
        'high': current_bar['y0_high'],
        'low': current_bar['y0_low'],
        'close': current_bar['y0_close'],
        'ema9': current_bar['y0_ema9'],
        'ema21': current_bar['y0_ema21'],
        'open_real': current_bar['y0_open_real'],
        'high_real': current_bar['y0_high_real'],
        'low_real': current_bar['y0_low_real'],
        'close_real': current_bar['y0_close_real']
        },
        {
        'date': 13,
        'open': current_bar['y1_open'],
        'high': current_bar['y1_high'],
        'low': current_bar['y1_low'],
        'close': current_bar['y1_close'],
        'ema9': current_bar['y1_ema9'],
        'ema21': current_bar['y1_ema21'],           
        'open_real': current_bar['y1_open_real'],
        'high_real': current_bar['y1_high_real'],
        'low_real': current_bar['y1_low_real'],
        'close_real': current_bar['y1_close_real']
        },
        {
        'date': 14,
        'open': current_bar['y2_open'],
        'high': current_bar['y2_high'],
        'low': current_bar['y2_low'],
        'close': current_bar['y2_close'],
        'ema9': current_bar['y2_ema9'],
        'ema21': current_bar['y2_ema21'],
        'open_real': current_bar['y2_open_real'],
        'high_real': current_bar['y2_high_real'],
        'low_real': current_bar['y2_low_real'],
        'close_real': current_bar['y2_close_real']
        },
        {
        'date': 15,
        'open': current_bar['y3_open'],
        'high': current_bar['y3_high'],
        'low': current_bar['y3_low'],
        'close': current_bar['y3_close'],
        'ema9': current_bar['y3_ema9'],
        'ema21': current_bar['y3_ema21'],
        'open_real': current_bar['y3_open_real'],
        'high_real': current_bar['y3_high_real'],
        'low_real': current_bar['y3_low_real'],
        'close_real': current_bar['y3_close_real']
        },     
        {
        'date': 16,
        'open': current_bar['y4_open'],
        'high': current_bar['y4_high'],
        'low': current_bar['y4_low'],
        'close': current_bar['y4_close'],
        'ema9': current_bar['y4_ema9'],
        'ema21': current_bar['y4_ema21'],
        'open_real': current_bar['y4_open_real'],
        'high_real': current_bar['y4_high_real'],
        'low_real': current_bar['y4_low_real'],
        'close_real': current_bar['y4_close_real']
        },    
        {
        'date': 17,
        'open': current_bar['y5_open'],
        'high': current_bar['y5_high'],
        'low': current_bar['y5_low'],
        'close': current_bar['y5_close'],
        'ema9': current_bar['y5_ema9'],
        'ema21': current_bar['y5_ema21'],
        'open_real': current_bar['y5_open_real'],
        'high_real': current_bar['y5_high_real'],
        'low_real': current_bar['y5_low_real'],
        'close_real': current_bar['y5_close_real']
        },    
        {
        'date': 18,
        'open': current_bar['y6_open'],
        'high': current_bar['y6_high'],
        'low': current_bar['y6_low'],
        'close': current_bar['y6_close'],
        'ema9': current_bar['y6_ema9'],
        'ema21': current_bar['y6_ema21'],
        'open_real': current_bar['y6_open_real'],
        'high_real': current_bar['y6_high_real'],
        'low_real': current_bar['y6_low_real'],
        'close_real': current_bar['y6_close_real']
        },    
        {
        'date': 19,
        'open': current_bar['y7_open'],
        'high': current_bar['y7_high'],
        'low': current_bar['y7_low'],
        'close': current_bar['y7_close'],
        'ema9': current_bar['y7_ema9'],
        'ema21': current_bar['y7_ema21'],
        'open_real': current_bar['y7_open_real'],
        'high_real': current_bar['y7_high_real'],
        'low_real': current_bar['y7_low_real'],
        'close_real': current_bar['y7_close_real']
        },    
        {
        'date': 20,
        'open': current_bar['y8_open'],
        'high': current_bar['y8_high'],
        'low': current_bar['y8_low'],
        'close': current_bar['y8_close'],
        'ema9': current_bar['y8_ema9'],
        'ema21': current_bar['y8_ema21'],
        'open_real': current_bar['y8_open_real'],
        'high_real': current_bar['y8_high_real'],
        'low_real': current_bar['y8_low_real'],
        'close_real': current_bar['y8_close_real']
        },         
        {
        'date': 21,
        'open': current_bar['y9_open'],
        'high': current_bar['y9_high'],
        'low': current_bar['y9_low'],
        'close': current_bar['y9_close'],
        'ema9': current_bar['y9_ema9'],
        'ema21': current_bar['y9_ema21'],
        'open_real': current_bar['y9_open_real'],
        'high_real': current_bar['y9_high_real'],
        'low_real': current_bar['y9_low_real'],
        'close_real': current_bar['y9_close_real']
        },         
        {
        'date': 22,
        'open': current_bar['y10_open'],
        'high': current_bar['y10_high'],
        'low': current_bar['y10_low'],
        'close': current_bar['y10_close'],
        'ema9': current_bar['y10_ema9'],
        'ema21': current_bar['y10_ema21'],
        'open_real': current_bar['y10_open_real'],
        'high_real': current_bar['y10_high_real'],
        'low_real': current_bar['y10_low_real'],
        'close_real': current_bar['y10_close_real']
        },                 
        {
        'date': 23,
        'open': current_bar['y11_open'],
        'high': current_bar['y11_high'],
        'low': current_bar['y11_low'],
        'close': current_bar['y11_close'],
        'ema9': current_bar['y11_ema9'],
        'ema21': current_bar['y11_ema21'], 
        'open_real': current_bar['y11_open_real'],
        'high_real': current_bar['y11_high_real'],
        'low_real': current_bar['y1_low_real'],
        'close_real': current_bar['y11_close_real']
        }]
    
    current_bar['current_pos'] = current_bar['x11_close']
    bars_for_evaluation = [x for x in current_bar['chart_data'] if x['date'] >= 12] 
    current_bar['is_short'] = is_short(current_bar['current_pos'], bars_for_evaluation, CURRENT_TARGET, CURRENT_STOP)
    current_bar['is_long'] = is_long(current_bar['current_pos'], bars_for_evaluation, CURRENT_TARGET, CURRENT_STOP)
    current_bar['current_date'] = current_date
    
    generated_bars.append(current_bar)
    
generated_bars_train_count = len(generated_bars_train)
generated_bars_test_count = len(generated_bars_test)
generated_bars_train_short_count = len([x for x in generated_bars_train if x['is_short'] == 1])
generated_bars_train_long_count = len([x for x in generated_bars_train if x['is_long'] == 1])

print(f'(Train)Generated Bars Count:{generated_bars_train_count}')
print(f'(Test)Generated Bars Count:{generated_bars_test_count}')
print(f'(Train)Generated Bars Short:{generated_bars_train_short_count} ({(generated_bars_train_short_count / generated_bars_train_count) * 100}%)')
print(f'(Train)Generated Bars Long:{generated_bars_train_long_count} ({(generated_bars_train_long_count / generated_bars_train_count) * 100}%)')
    

In [ ]:
if True:
    TOOLS = "pan,wheel_zoom,box_zoom,reset,save"
    generated_bars_to_visualize = [x for x in generated_bars_train if x['is_long'] == 1][10]
    df = pd.DataFrame(generated_bars_to_visualize['chart_data'])

    df['previous_high']=generated_bars_to_visualize['previous_high']
    df['previous_low']=generated_bars_to_visualize['previous_low']
    df['previous_close']=generated_bars_to_visualize['previous_close']
    df['previous_open']=generated_bars_to_visualize['previous_open']

    df['previous_high_real']=generated_bars_to_visualize['previous_high_real']
    df['previous_low_real']=generated_bars_to_visualize['previous_low_real']
    df['previous_close_real']=generated_bars_to_visualize['previous_close_real']
    df['previous_open_real']=generated_bars_to_visualize['previous_open_real']
    df['current_pos']=generated_bars_to_visualize['current_pos']
    
    current_start = generated_bars_to_visualize['x5_close_real']
    current_date = generated_bars_to_visualize['current_date']
    current_time = generated_bars_to_visualize['current_time']

    p = figure(x_axis_type="linear", tools=TOOLS, width=1000, title = f'WDO Candlestick, start:{current_start}, date:{current_date}, time:{current_time}')
    p.xaxis.major_label_orientation = pi/4
    p.grid.grid_line_alpha=0.3
    w = 12*60*60*1000 # half day in ms

    inc = df.close > df.open
    dec = df.open > df.close

    p.segment(df.date, df.high, df.date, df.low, color="black")
    p.vbar(df.date[inc], 0.8, df.open[inc], df.close[inc], fill_color="#D5E1DD", line_color="black")
    p.vbar(df.date[dec], 0.8, df.open[dec], df.close[dec], fill_color="#F2583E", line_color="black")

    p.line(df.date, df.ema9, line_color="#00FF00", line_width = 2)
    p.line(df.date, df.ema21, line_color="#FFFF00", line_width = 2)
    p.line(df.date, df.previous_high, line_color="#FF0000", line_width = 2)
    p.line(df.date, df.previous_close, line_color="#FF0000", line_width = 2)
    p.line(df.date, df.previous_open, line_color="#FF0000", line_width = 2)
    p.line(df.date, df.previous_low, line_color="#FF0000", line_width = 2)
    p.line(df.date, df.current_pos, line_color="#000000", line_width = 2)

    show(p)

In [ ]:
train_output_file_name = os.path.join(f"{CURRENT_ASSET}.{CURRENT_TIMEFRAME}.{int(CURRENT_TARGET * 100)}.{int(CURRENT_STOP * 100)}.train.pickle")
train_output_full_file_name = os.path.join('..','00.data','output',train_output_file_name)
joblib.dump(generated_bars_train, train_output_full_file_name, True)

test_output_file_name = os.path.join(f"{CURRENT_ASSET}.{CURRENT_TIMEFRAME}.{int(CURRENT_TARGET * 100)}.{int(CURRENT_STOP * 100)}.test.pickle")
test_output_full_file_name = os.path.join('..','00.data','output',test_output_file_name)
joblib.dump(generated_bars_test, test_output_full_file_name, True)

test_output_file_name = os.path.join(f"{CURRENT_ASSET}.{CURRENT_TIMEFRAME}.{int(CURRENT_TARGET * 100)}.{int(CURRENT_STOP * 100)}.total.pickle")
test_output_full_file_name = os.path.join('..','00.data','output',test_output_file_name)
joblib.dump(generated_bars_train + generated_bars_test, test_output_full_file_name, True)

parameter_file_name = test_output_file_name = os.path.join(f"{CURRENT_ASSET}.{CURRENT_TIMEFRAME}.{int(CURRENT_TARGET * 100)}.{int(CURRENT_STOP * 100)}.parameters.pickle")
parameter_full_file_name = os.path.join('..','00.data','output',test_output_file_name)
parameters = {
    "MINIMUM_TIME" : MINIMUM_TIME,
    "MAXIMUM_TIME" : MAXIMUM_TIME,
    "MINIMUM_DATE_DATAFRAME" : MINIMUM_DATE_DATAFRAME,
    "MINIMUM_DATE_TRADE" : MINIMUM_DATE_TRADE,
    "MAX_TRAIN_DATE" : MAX_TRAIN_DATE,
    "CURRENT_TARGET" : CURRENT_TARGET,
    "CURRENT_STOP" : CURRENT_STOP,
    "CURRENT_ASSET" : CURRENT_ASSET,
    "CURRENT_TIMEFRAME" : CURRENT_TIMEFRAME,
    "CURRENT_DAILY_FILE_CSV" : CURRENT_DAILY_FILE_CSV,
    "CURRENT_5MIN_FILE_CSV" : CURRENT_5MIN_FILE_CSV,
    "CURRENT_Y_COLUMN_SHORT" : 'is_short',
    "CURRENT_Y_COLUMN_LONG" : 'is_long',
    "CURRENT_X_COLUMNS_FULL" : ['previous_high', 'previous_close', 'previous_low' , 'previous_open', 'x0_ema9' , 'x0_ema21', 'x0_open', 'x0_high', 'x0_low', 'x0_close', 'x0_volume', 'x1_ema9', 'x1_ema21', 'x1_open', 'x1_high', 'x1_low', 'x1_close', 'x1_volume', 'x2_ema9', 'x2_ema21', 'x2_open', 'x2_high' , 'x2_low' , 'x2_close', 'x2_volume', 'x3_ema9' , 'x3_ema21', 'x3_open' , 'x3_high' , 'x3_low' , 'x3_close', 'x3_volume', 'x4_ema9' , 'x4_ema21', 'x4_open' , 'x4_high' , 'x4_low' , 'x4_close', 'x4_volume', 'x5_ema9' , 'x5_ema21', 'x5_open' , 'x5_high' , 'x5_low' , 'x5_close', 'x5_volume', 'x6_ema9' , 'x6_ema21', 'x6_open' , 'x6_high' , 'x6_low' , 'x6_close', 'x6_volume', 'x7_ema9' , 'x7_ema21', 'x7_open' , 'x7_high' , 'x7_low' , 'x7_close', 'x7_volume', 'x8_ema9' , 'x8_ema21', 'x8_open' , 'x8_high' , 'x8_low' , 'x8_close', 'x8_volume', 'x9_ema9' , 'x9_ema21', 'x9_open' , 'x9_high' , 'x9_low' , 'x9_close', 'x9_volume', 'x10_ema9' , 'x10_ema21', 'x10_open' , 'x10_high' , 'x10_low' , 'x10_close', 'x10_volume', 'x11_ema9' , 'x11_ema21', 'x11_open' , 'x11_high' , 'x11_low' , 'x11_close', 'x11_volume'],
    "CURRENT_X_COLUMNS" : ['x0_ema9' , 'x0_ema21', 'x0_open', 'x0_high', 'x0_low', 'x0_close', 'x0_volume', 'x1_ema9', 'x1_ema21', 'x1_open', 'x1_high', 'x1_low', 'x1_close', 'x1_volume', 'x2_ema9', 'x2_ema21', 'x2_open', 'x2_high' , 'x2_low' , 'x2_close', 'x2_volume', 'x3_ema9' , 'x3_ema21', 'x3_open' , 'x3_high' , 'x3_low' , 'x3_close', 'x3_volume', 'x4_ema9' , 'x4_ema21', 'x4_open' , 'x4_high' , 'x4_low' , 'x4_close', 'x4_volume', 'x5_ema9' , 'x5_ema21', 'x5_open' , 'x5_high' , 'x5_low' , 'x5_close', 'x5_volume', 'x6_ema9' , 'x6_ema21', 'x6_open' , 'x6_high' , 'x6_low' , 'x6_close', 'x6_volume', 'x7_ema9' , 'x7_ema21', 'x7_open' , 'x7_high' , 'x7_low' , 'x7_close', 'x7_volume', 'x8_ema9' , 'x8_ema21', 'x8_open' , 'x8_high' , 'x8_low' , 'x8_close', 'x8_volume', 'x9_ema9' , 'x9_ema21', 'x9_open' , 'x9_high' , 'x9_low' , 'x9_close', 'x9_volume', 'x10_ema9' , 'x10_ema21', 'x10_open' , 'x10_high' , 'x10_low' , 'x10_close', 'x10_volume', 'x11_ema9' , 'x11_ema21', 'x11_open' , 'x11_high' , 'x11_low' , 'x11_close', 'x11_volume']
}

joblib.dump(parameters, parameter_full_file_name, True)